In [ ]:
# Importamos las librerías que vamos a necesitar
import re
import requests
from bs4 import BeautifulSoup
import csv
import time
import unicodedata

# Función para eliminar tildes        
def elimina_tildes(cadena):
    s = ''.join((c for c in unicodedata.normalize('NFD',cadena) if unicodedata.category(c) != 'Mn'))
    return s

# Creamos el documento .csv para guardar los campos de interés
with open("Standards.csv","w", encoding="utf-8") as csvfile:
    spamwriter=csv.writer(csvfile, delimiter=';')  
    # Cabecera
    spamwriter.writerow(['CODIGO','DESCRIPCION', 'EMPRESA', 'CODIGO_NB', 'PAIS_NB', 'TELEFONO_NB', 'EMAIL_NB', 'WEB_NB'])
    cnt=0
    Datos=[]
   
    # Obtenemos los datos
    #Calculamos el tiempo de respuesta para introducir retrasos en las peticiones consecutivas
    for term in ["web scraping", "web crawling", "scrape this site"]:
        t0 = time.time()
        r = requests.get("https://ec.europa.eu/growth/tools-databases/nando/index.cfm?fuseaction=cp.hs&cpr=Y", params=dict(query=term))
       
    ## Calculo final de la estimación del tiempo de respuesta en segundos
    response_delay = time.time() - t0
   
    # Marcamos la url base para los enlaces siguientes extraidos de la web
    url_base = 'https://ec.europa.eu/growth/tools-databases/nando/'
   
    # Listas para guardar los campos de interés
    Codigo=[]
    Descripcion=[]
    Empresa=[]
    nb=[]
    Country=[]
    Phone=[]
    Email=[]
    Web=[]
   
    # Cargamos la primera página, donde se encuentran el listado de normas armonizadas.
    page = requests.get('https://ec.europa.eu/growth/tools-databases/nando/index.cfm?fuseaction=cp.hs&cpr=Y')
    soup = BeautifulSoup(page.content)
   
    contador=-1
   
    # Recorremos la lista de normas:
    for i in soup.find_all('a',{'class':'list'}):
        # Cada norma introducimos un tiempo de espera para espaciar nuestras peticiones
        # Espera de 10 veces el tiempo de respuesta
        time.sleep(10 * response_delay)
       
        #####################
        ### CAMPO CODIGO ###
        #####################
        codigo = i.string  # Valor a introducir en el campo: Codigo
       
        ###########################
        ### CAMPO DESCRIPCION ###
        ###########################
   
        cod_simple = re.findall('[A-Z]+ [0-9]*[-]*[0-9]*[:]*[0-9]*[/]*[+]*[A-Z]*[:]*[0-9]*', codigo)        
        if cod_simple[0] == 'EN ISO':
            cod_simple = re.findall('[A-Z]+ [A-Z]* [0-9]*[-]*[0-9]*[:]*[0-9]*[/]*[+]*[A-Z]*[:]*[0-9]*', codigo)
        descripcion = soup.find_all(string=re.compile('\t'+cod_simple[0]))
       
        try:
            descripcion =(descripcion[0].strip())  # Valor a introducir en el campo: Descripcion
        except IndexError:  
            descripcion ="---"
    #   print(descripcion)
         
        if contador==1:
            break
        else:
            contador=contador+1
   
        # Cargamos la segunda página, de esta página no estraeremos datos con los que rellenar campos
        # pero muestra un listado de los NB que pueden realizar la notificación de la norma que estamos
        # tratando.
        page2 = requests.get(url_base+i.get('href'))
        soup2 = BeautifulSoup(page2.content)
       
        # Recorremos el listado de NB de la norma
        for j in soup2.find_all('a',{'class':'list'}):
            #Para cada empresa introducimos un tiempo de espera para espaciar nuestras peticiones
            #Espera de 2 veces el tiempo de respuesta
            time.sleep(2 * response_delay)
           
            #Para cada elemento de la segunda página, es decir, para cada NB cargamos una página
            #en la que están sus datos.
            page3 = requests.get(url_base+j.get('href'))
            soup3 = BeautifulSoup(page3.content)
   
            #############################
            ### CAMPO EMPRESA y WEB ###
            #############################
            for k in soup3.find_all('strong'):
                empresa=(k.string) # Valor a introducir en el campo: Empresa
                empresa=elimina_tildes(empresa)
                Empresa.append(empresa)
               
                web=soup3.find('a',{'target':'_blank'})  # Valor a introducir en el campo: Web_NB
                if web:
                    # print(web.get('href'))
                    Web.append(web.get('href'))
                else:
                    Web.append("---")
       
            ##############################################
            ### CAMPOS NB, PAIS, TELEFONO, EMAIL       ###
            ##############################################
            NB = soup3.find_all(string=re.compile("Notified Body number :"))
            aux = NB[0].replace('Notified Body number :', '')
            NB = aux.strip() #Valor a introducir en el campo: Codigo_NB
            nb.append(NB)
   
            country = soup3.find_all(string=re.compile("Country :"))
            aux = country[0].replace('Country :', '')
            country = aux.strip() #Valor a introducir en el campo: Pais_NB
            Country.append(country)
   
            phone = soup3.find_all(string=re.compile("Phone :"))
            aux = phone[0].replace('Phone :', '')
            phone = aux.strip() #Valor a introducir en el campo: Telefono_NB
            Phone.append(phone)
   
            email = soup3.find_all(string=re.compile("Email :"))
            aux = email[0].replace('Email :', '')
            email = aux.strip() #Valor a introducir en el campo: Email_NB
            Email.append(email)
           
            # Almacenamos el Código y la Descripción
            Codigo.append(codigo)
            Descripcion.append(descripcion)
            
            #############################
            ### GUARDAMOS LOS DATOS ###
            #############################  
            # Montamos los datos en una fila y lo guardamos en el csv
            Rows=[Codigo[cnt], Descripcion[cnt], Empresa[cnt], nb[cnt], Country[cnt], Phone[cnt], Email[cnt], Web[cnt]]
            Datos.append(Rows)
            # print(Datos[cnt])
            spamwriter.writerow(Datos[cnt])
            
            #Imprimimos un aviso cada 25 registros para saber que el proceso continua. 
            if cnt%25 == 0:
                print("Guardando fila... " + str(cnt))
            cnt=cnt+1

csvfile.close()
print("Proceso finalizado. \nSe han guardado "+str(cnt)+" registros")
 